In [1]:
%%time

#import in all libraries and methods
%run -i 'myimports.py'
%run -i 'datagathering.py'
%run -i 'nlp_processing.py' #imports packages and methods
%run -i 'timestamp_generation.py'

main = "https://www.happyscribe.com"                
url="/public/lex-fridman-podcast-artificial-intelligence-ai"
podcast_host = "Lex Fridman"
html_location = "Archive/happyscribe_Lex_Fridman_html.txt"



CPU times: user 15.9 s, sys: 1.37 s, total: 17.3 s
Wall time: 18.2 s


In [1]:
%%time
import pymongo
from pymongo import MongoClient
import urllib.request
import urllib.parse

username = urllib.parse.quote_plus('Empyr3an')
password = urllib.parse.quote_plus("B4ldr1c7@1")

cluster = pymongo.MongoClient("mongodb+srv://{}:{}@cluster0.4pec2.mongodb.net/Checkra?retryWrites=true&w=majority".format(username, password))
db = cluster["podcasts"]
collection = db["lex"]

CPU times: user 128 ms, sys: 58.6 ms, total: 187 ms
Wall time: 385 ms


In [13]:
for arr in collection.find({},{"places":1}):
    print(arr)

{'_id': ObjectId('5fe6859cc9a88b14eadc764f'), 'places': ['Tehran', 'Vancouver', 'Africa', 'Bethesda', 'London', 'SIM City', 'Ethiopia', 'Proteau Ajai', 'AZ', 'Katrina', 'Mars', 'D.C.', 'Najai', 'Las Vegas', 'Earth', 'New York', 'Canterbury', 'the United States', 'Abu Ghraib', 'Washington', 'Exito', 'Southern California', 'Hong Kong', 'China', 'North Philly', 'Silicon Valley', 'Venezuela', 'Koven', 'Louisiana', 'Tensas', 'Iran', 'Ethereum', 'Alpha Centauri', 'Shenzhen', 'Congo', 'AI', 'South America', 'Turing', 'Cisco', 'Sofia', 'Dharma', 'Philadelphia', 'California', 'Addis Ababa', 'Winton', 'Giannakou', 'Seattle', 'South China', 'Lithuania', 'Austin', 'DC', 'Japan', 'Tawang', 'US', 'Eastern Europe', 'Russia', 'Germany', 'KADEK', 'Prague', 'Bitcoin', 'Tenzer', 'Texas', 'Australia', 'Silicon Alley', 'Asia', 'Alibaba', 'Europe', 'New Zealand', 'the Solaris Superintelligent Ocean', 'Iraq', 'Iceland', 'Poland', 'St. Petersburg', 'Baulderstone']}
{'_id': ObjectId('5fe685a7c9a88b14eadc7650')

In [ ]:
print(list(collection.find({"guest":"Ben Goertzel"})))

In [50]:
%%time
print("bro")
doc = nlp(text_fix(open("3Lex/#103|Ben_Goertzel|Artificial_General_Intelligence.txt").read()))
print(doc.user_data.keys())

bro
dict_keys(['entis', 'keywords', 'summary', 'subtopics'])
CPU times: user 13.3 s, sys: 4.06 s, total: 17.4 s
Wall time: 18.2 s


In [5]:
ents = doc.user_data["entis"]

In [7]:
places = list(set([e[0] for e in ents if e[1]=="LOC" or e[1]=="GPE"]))

In [9]:
type(places)

list

In [51]:
def is_person(name):
    try:
        result = wikipedia.search(name)[0]
        if len(result.split(" "))>1 and name in result:
            return(result, True)
        else:
            return (name, False)
    except:
        return("none", False)
def keep_ents(doc):
    books, people, places = [], [], []
    ents = [e for e in doc.user_data["entis"] if e[0].replace(".","").lower()!="phd"]
    
    doc.user_data["places"] = list(set([e[0] for e in ents if e[1]=="LOC" or e[1]=="GPE"]))
    
    #parallel processing to verify people and get full names from wikipedia
    people = list(set([e[0] for e in ents if e[1]=="PERSON"]))
    finalpeople = []
    with concurrent.futures.ThreadPoolExecutor(max_workers = 30) as executor:
        result = [executor.submit(is_person, p) for p in people]
    for future in concurrent.futures.as_completed(result):
#         print(future.result())
        if future.result()[1]==True:
            finalpeople.append(future.result()[0])
    doc.user_data["people"] = finalpeople
    
    #parallel processing to verify books
    books = list(set([e[0] for e in ents if e[1]=="WORK_OF_ART"]))
    allbooks = []
    with concurrent.futures.ThreadPoolExecutor(max_workers = 30) as executor:
        result = [executor.submit(is_book, book) for book in books]
    for future in concurrent.futures.as_completed(result):
        if future.result()[1]==True:
            allbooks.append(future.result()[0])
    doc.user_data["books"] =allbooks
    return doc
doc = keep_ents(doc)

In [53]:
doc.user_data["subtopics"]

[['So he had a very broad view of intelligence going beyond the human and into what i would call, open ended superintelligence that the solaris superintelligent ocean was intelligent in some ways more generally intelligent than people, but in a complex and confusing way so that human beings could never quite connect to it.',
  'Naturally occurring intelligence then philip k. dick, who, ultimately my friend and philip k. dick is one of the things that brought me together with david hanson, my collaborator on robotics project.',
  'And there was not that much science fiction to watch on tv at that stage that got me into reading the whole literature of science fiction, from the beginning of the previous century and until that time.',
  'The show, sponsored by a master class sign up, a master class that looks to get a discount and to support this podcast.',
  'Please consider supporting this podcast by going to jordan harbinger, that complex and signing up a master class'],
 ['Is it the hu

In [ ]:
type(collection.insertOne(doc.user_data))

In [ ]:
for topic in doc.user_data["subtopics"]:
    print(" ".join(topic[:2]), "\n")

In [ ]:
update_transcripts(html_location, main)

In [ ]:
%%time
# %run -i 'myimports.py'
#convert file of podcasts to list of docs and docbin and write to disk
# %run -i 'nlp_implementation.py'
podcast_name = "3Lex"
# print(contractions.fix("hey what's up"))

os.listdir(podcast_name)
onlyfiles = folder_to_filelist(podcast_name) #get list of all files in foldername
# doc_bin = DocBin(store_user_data=True) #docbin container for serialization
docs = [] #list of docs 
print("starting")
for doc, name in nlp.pipe(onlyfiles, as_tuples=True): #piping all collection of docs to make doclist and docbin
    #each doc contains hostname, guest, title, entities mentioned, and summary
    name = re.split("[\|]",name)
    name=name[1:] if name[0][1:].isdigit() else name # store name of guest and topic, add to doc user data
    
    doc.user_data["host"] = podcast_host
    doc.user_data["guest"]= str(name[0]).replace("_"," ")
    doc.user_data["title"]= str(name[1][:-4]).replace("_"," ")
    
    docs.append(doc)
    collection.insert_one(doc.user_data)
#     doc_bin.add(doc) #add doc to list and bin
    
# with open("cucumber/Lexnumbered.bin", "wb") as f: #write bytestream to first_doc.bin
#     f.write(doc_bin.to_bytes())
print("done")

In [ ]:
# for doc in collection.find({"host":"Lex Fridman"}):
#     print(doc)

In [ ]:
%%time
#loading in previous bytestreams, needs 2 since 2 podcast folders edit later
with open("cucumber/Lex.bin", "rb") as w: 
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))
# with open("cucumber/first_doc.bin", "rb") as w: #loading in
#     new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
# docs1 += list(new_docs1.get_docs(nlp.vocab))
for doc in docs1:
    print(doc.user_data["title"], doc.user_data["guest"])
# print(docs1[5].user_data["books"])

In [ ]:
#for each category of ents, generate csv files for ents corresponding to people

att_to_csv(docs1, "people") 
# att_to_csv(docs1, "places")
# att_to_csv(docs1, "books")
# print(sorted(set([item for sublist in docs1 for item in sublist.user_data["people"]])))

In [ ]:
#generate megagraph for book ents
edges = pd.read_csv("booksEdges.csv", sep=',').drop("id", axis=1).values.tolist()
nodes = pd.read_csv('booksNodes.csv',sep=',').set_index("id").to_dict("index")

source = [nodes.get(e[0])["name"] for e in edges]
target = [nodes.get(e[1])["name"] for e in edges]
kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':np.ones(len(source))})
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

#following are other example graphviews

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['target']=="Bible"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['source']=="Matthew Johnson"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[(kg_df['source']=="Matthew Johnson" )| (kg_df['source']=="Ryan Hall")], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[(kg_df['target']=="Crime and Punishment" )| (kg_df['target']=="Atlas Shrugged")], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [48]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
 
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
#     print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))

# let's begin
generate_summary( "msft.txt", 1)

He was a very astute analyst of, you know, how the human mind constructs the illusion of itself, how it constructs the illusion of free will, how it constructs values like like good and evil out of its own, you know, desire to maintain and advance its own organism
So i think that book for the last robock, which i think i read when it first came out, i would have been 12 years old or something
So i think if you look at the evolution, humanity is shaped both by individual selection and what biologists would call group selection, like tribe level selection
should send people out to every little village in remotest africa, south america, and explain to everyone what technology was going to bring the next few decades and the choice that we had about how to use it and let everyone and the whole planet vote about whether we should develop, you know, If we look at i know it is hard to look at humans in the aggregate, but do you think overall humans are good?
Summarize Text: 
 So i think if you

In [44]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /ilab/users/hs884/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True